### Import libraries

In [1]:
from IPython.core.display import display
from src.utils.preprocessing import classic_preprocessing
from src.utils.get_data import import_data, split_experts
from src.utils.train import hyperparameter_tuning_cv
from src.utils.config import *
import pandas as pd
import numpy as np

In [2]:
%load_ext autoreload
%autoreload 2

### Load Data

In [3]:
DATA_PATH = '../../../data'
X_fine, y_fine = import_data(DATA_PATH, segmentation_type='fine',
                             drop_user_features=False,
                             drop_expert=True)

In [4]:
# For each expert separately
X_e, y_e = import_data(DATA_PATH, segmentation_type='fine',
                       drop_user_features=False,
                       drop_expert=False)

In [5]:
display(X_fine.head())
display(y_fine.head())

EEPD50_100  EEPD100_150  \
subject                              file_id                            
008ba489-31ad-44d8-856b-fcf72369dc46 0               6.0          6.0   
                                     1               3.0          3.0   
                                     2               4.0          4.0   
                                     3               4.0          4.0   
                                     4               4.0          4.0   

                                              EEPD150_200  EEPD200_250  \
subject                              file_id                             
008ba489-31ad-44d8-856b-fcf72369dc46 0                5.0          4.0   
                                     1                3.0          4.0   
                                     2                3.0          4.0   
                                     3                4.0          4.0   
                                     4                5.0          4.0   

                                              EEPD250_300  EEPD300_350  \
subject                              file_id                             
008ba489-31ad-44d8-856b-fcf72369dc46 0                5.0          5.0   
                                     1                4.0          5.0   
                                     2                3.0          3.0   
                                     3                4.0          5.0   
                                     4                4.0          5.0   

                                              EEPD350_400  EEPD400_450  \
subject                              file_id                             
008ba489-31ad-44d8-856b-fcf72369dc46 0                6.0          6.0   
                                     1                4.0          4.0   
                                     2                4.0          4.0   
                                     3                3.0          4.0   
                                     4                3.0          4.0   

                                              EEPD450_500  EEPD500_550  ...  \
subject                              file_id                            ...   
008ba489-31ad-44d8-856b-fcf72369dc46 0                5.0          6.0  ...   
                                     1                4.0          3.0  ...   
                                     2                4.0          4.0  ...   
                                     3                4.0          3.0  ...   
                                     4                4.0          3.0  ...   

                                              PSD_225-425  PSD_450-550  \
subject                              file_id                             
008ba489-31ad-44d8-856b-fcf72369dc46 0           0.398299     0.026407   
                                     1           0.230052     0.086242   
                                     2           0.105747     0.180344   
                                     3           0.235517     0.121967   
                                     4           0.230394     0.119689   

                                              PSD_1325-1600  PSD_1600-1900  \
subject                              file_id                                 
008ba489-31ad-44d8-856b-fcf72369dc46 0             0.013356       0.033366   
                                     1             0.023806       0.008824   
                                     2             0.034701       0.006258   
                                     3             0.061882       0.002742   
                                     4             0.069418       0.002717   

                                              PSD_2500-2900  PSD_3100-3700  \
subject                              file_id                                 
008ba489-31ad-44d8-856b-fcf72369dc46 0             0.043195       0.010385   
                                     1             0.056102       0.011429   
                                     2       

Label
subject                              file_id       
008ba489-31ad-44d8-856b-fcf72369dc46 0          1.0
                                     1          1.0
                                     2          1.0
                                     3          1.0
                                     4          1.0

### Preprocessing

In [6]:
X_fine = classic_preprocessing(X_fine)

In [7]:
# Save the expert feature for split
expert = X_e['Expert'].copy()
X_e = classic_preprocessing(X_e)
X_e['Expert'] = expert.values

### Split expert models

In [8]:
X_e_1, y_e_1, X_e_2, y_e_2, X_e_3, y_e_3 = split_experts(X_e, y_e)

### Grid search

In [ ]:
DECISION_METRIC = 'roc_auc_score'

#### 1. Logistic regression

In [9]:
log_results = hyperparameter_tuning_cv(model='logistic', data=X_fine, labels=y_fine.Label, cv_k=5,
                                       params=LOGISTIC_PARAMS)

display(log_results)

f1_score  roc_auc_score  accuracy_score  \
max_iter oversampling                                            
10000    True          0.402198       0.574983        0.622634   
         False         0.157118       0.530696        0.732477   
100000   True          0.402198       0.574983        0.622634   
         False         0.157118       0.530696        0.732477   

                       explained_variance_score  
max_iter oversampling                            
10000    True                         -0.873100  
         False                        -0.108863  
100000   True                         -0.873100  
         False                        -0.108863

In [10]:
# Best model parameters

best_log = log_results.iloc[[log_results.reset_index()[DECISION_METRIC].idxmax()]]

display(best_log)

,,f1_score,roc_auc_score,accuracy_score,explained_variance_score
max_iter,oversampling,,,,
10000,True,0.402198,0.574983,0.622634,-0.8731


In [11]:
# for each expert
log_results_1 = hyperparameter_tuning_cv(model='logistic', data=X_e_1, labels=y_e_1, cv_k=5,
                                         params=LOGISTIC_PARAMS)
log_results_2 = hyperparameter_tuning_cv(model='logistic', data=X_e_2, labels=y_e_2, cv_k=5,
                                         params=LOGISTIC_PARAMS)
log_results_3 = hyperparameter_tuning_cv(model='logistic', data=X_e_3, labels=y_e_3, cv_k=5,
                                         params=LOGISTIC_PARAMS)

In [12]:
# Best model parameters

best_log_1 = log_results_1.iloc[[
    log_results_1.reset_index()[DECISION_METRIC].idxmax()]]
best_log_2 = log_results_2.iloc[[
    log_results_2.reset_index()[DECISION_METRIC].idxmax()]]
best_log_3 = log_results_3.iloc[[
    log_results_3.reset_index()[DECISION_METRIC].idxmax()]]

display(best_log_1, best_log_2, best_log_3)

,,f1_score,roc_auc_score,accuracy_score,explained_variance_score
max_iter,oversampling,,,,
10000,True,0.366321,0.647065,0.736791,-1.001243


,,f1_score,roc_auc_score,accuracy_score,explained_variance_score
max_iter,oversampling,,,,
10000,True,0.29471,0.550656,0.645144,-1.229032


,,f1_score,roc_auc_score,accuracy_score,explained_variance_score
max_iter,oversampling,,,,
10000,True,0.642027,0.659811,0.660414,-0.362885


#### 2. Linear Discriminant Analysis

In [13]:
lda_results = hyperparameter_tuning_cv(model='lda', data=X_fine, labels=y_fine.Label, cv_k=5,
                                       params=LDA_PARAMS)

display(lda_results)

,f1_score,roc_auc_score,accuracy_score,explained_variance_score
oversampling,,,,
True,0.402724,0.576162,0.625392,-0.862397
False,0.166116,0.532169,0.731526,-0.123842


In [14]:
# Best model parameters

best_lda = lda_results.iloc[[lda_results.reset_index()[DECISION_METRIC].idxmax()]]

display(best_lda)

,f1_score,roc_auc_score,accuracy_score,explained_variance_score
oversampling,,,,
True,0.402724,0.576162,0.625392,-0.862397


In [15]:
# For each expert
lda_results_1 = hyperparameter_tuning_cv(model='lda', data=X_e_1, labels=y_e_1, cv_k=5,
                                         params=LDA_PARAMS)
lda_results_2 = hyperparameter_tuning_cv(model='lda', data=X_e_2, labels=y_e_2, cv_k=5,
                                         params=LDA_PARAMS)
lda_results_3 = hyperparameter_tuning_cv(model='lda', data=X_e_3, labels=y_e_3, cv_k=5,
                                         params=LDA_PARAMS)

In [16]:
best_lda_1 = lda_results_1.iloc[[
    lda_results_1.reset_index()[DECISION_METRIC].idxmax()]]
best_lda_2 = lda_results_2.iloc[[
    lda_results_2.reset_index()[DECISION_METRIC].idxmax()]]
best_lda_3 = lda_results_3.iloc[[
    lda_results_3.reset_index()[DECISION_METRIC].idxmax()]]

display(best_lda_1, best_lda_2, best_lda_3)

,f1_score,roc_auc_score,accuracy_score,explained_variance_score
oversampling,,,,
True,0.359445,0.643375,0.724473,-1.070413


,f1_score,roc_auc_score,accuracy_score,explained_variance_score
oversampling,,,,
True,0.291674,0.54846,0.645144,-1.233195


,f1_score,roc_auc_score,accuracy_score,explained_variance_score
oversampling,,,,
True,0.640336,0.659293,0.660138,-0.363984


#### 3. K-nearest Neighbors

In [17]:
knn_results = hyperparameter_tuning_cv(model='knn', data=X_fine, labels=y_fine.Label, cv_k=5,
                                       params=KNN_PARAMS)

display(knn_results)

f1_score  roc_auc_score  accuracy_score  \
n_neighbors oversampling                                            
1           True          0.538261       0.683665        0.662863   
            False         0.515077       0.668555        0.746172   
2           True          0.528326       0.675809        0.700238   
            False         0.323353       0.586013        0.756633   
3           True          0.508429       0.651581        0.584118   
            False         0.431194       0.621831        0.739515   
4           True          0.510313       0.656332        0.621969   
            False         0.290474       0.573358        0.750071   
5           True          0.495399       0.634570        0.544746   
            False         0.387260       0.605420        0.748264   
6           True          0.499344       0.642232        0.578887   
            False         0.252190       0.560634        0.745602   
7           True          0.485318       0.620226        0.515644   
            False         0.330488       0.584059        0.745982   
8           True          0.490013       0.629267        0.547599   
            False         0.211966       0.547526        0.740086   
9           True          0.479705       0.611329        0.494817   
            False         0.273073       0.563421        0.739135   
10          True          0.480628       0.615864        0.518973   
            False         0.187402       0.542011        0.740561   
11          True          0.472081       0.599816        0.474085   
            False         0.244725       0.556350        0.740846   
12          True          0.476706       0.608947        0.500238   
            False         0.163097       0.535999        0.739515   
13          True          0.466911       0.591658        0.459439   
            False         0.212515       0.547146        0.739039   
14          True          0.471130       0.600587        0.485402   
            False         0.144381       0.530611        0.737233   
15          True          0.465740       0.588807        0.450689   
            False         0.186899       0.540420        0.737993   

                          explained_variance_score  
n_neighbors oversampling                            
1           True                         -0.527275  
            False                        -0.288759  
2           True                         -0.475715  
            False                        -0.072149  
3           True                         -0.633944  
            False                        -0.289575  
4           True                         -0.644865  
            False                        -0.092480  
5           True                         -0.642629  
            False                        -0.194903  
6           True                         -0.675038  
            False                        -0.091073  
7           True                         -0.639176  
            False                        -0.160332  
8           True                         -0.682526  
            False                        -0.098110  
9           True                         -0.617721  
            False                        -0.160118  
10          True                         -0.681177  
            False                        -0.072435  
11          True                         -0.608705  
            False                        -0.121379  
12          True                         -0.658779  
            False                        -0.059981  
13          True                         -0.599336  
            False                        -0.106807  
14          True                         -0.656389  
            False                        -0.062013  
15          True                         -0.574382  
            False                        -0.092197

In [18]:
# Best model parameters

best_knn = knn_results.iloc[[knn_results.reset_index()[DECISION_METRIC].idxmax()]]

display(best_knn)

,,f1_score,roc_auc_score,accuracy_score,explained_variance_score
n_neighbors,oversampling,,,,
1,True,0.538261,0.683665,0.662863,-0.527275


In [19]:
knn_results_1 = hyperparameter_tuning_cv(
    model='knn', data=X_e_1, labels=y_e_1, cv_k=5, params=KNN_PARAMS)
knn_results_2 = hyperparameter_tuning_cv(
    model='knn', data=X_e_2, labels=y_e_2, cv_k=5, params=KNN_PARAMS)
knn_results_3 = hyperparameter_tuning_cv(
    model='knn', data=X_e_3, labels=y_e_3, cv_k=5, params=KNN_PARAMS)

In [22]:
display(knn_results_1, knn_results_2, knn_results_3)

f1_score  roc_auc_score  accuracy_score  \
n_neighbors oversampling                                            
1           True          0.464680       0.754597        0.740357   
            False         0.565508       0.747747        0.872285   
2           True          0.488683       0.754113        0.783468   
            False         0.376124       0.618609        0.877472   
3           True          0.393392       0.709754        0.628525   
            False         0.436504       0.652852        0.869368   
4           True          0.413733       0.722199        0.671961   
            False         0.254010       0.572802        0.867099   
5           True          0.369463       0.693009        0.562075   
            False         0.338336       0.604841        0.869692   
6           True          0.384681       0.704501        0.605835   
            False         0.179824       0.548555        0.862237   
7           True          0.349909       0.670573        0.519611   
            False         0.235414       0.566080        0.862237   
8           True          0.367977       0.690570        0.561102   
            False         0.130779       0.533276        0.858671   
9           True          0.343029       0.663709        0.493679   
            False         0.165642       0.543531        0.858671   
10          True          0.354973       0.677714        0.524473   
            False         0.090358       0.522280        0.857374   
11          True          0.334899       0.654257        0.464182   
            False         0.123706       0.531133        0.857699   
12          True          0.346171       0.667992        0.498865   
            False         0.061327       0.514953        0.856402   
13          True          0.326620       0.642289        0.442139   
            False         0.079898       0.518596        0.854781   
14          True          0.336940       0.656215        0.473258   
            False         0.046657       0.510892        0.855105   
15          True          0.320060       0.632360        0.428201   
            False         0.061583       0.514512        0.855429   

                          explained_variance_score  
n_neighbors oversampling                            
1           True                         -0.791519  
            False                        -0.026441  
2           True                         -0.604962  
            False                         0.095853  
3           True                         -1.170149  
            False                        -0.018005  
4           True                         -1.071199  
            False                         0.037671  
5           True                         -1.228692  
            False                         0.024091  
6           True                         -1.201792  
            False                         0.019912  
7           True                         -1.266890  
            False                        -0.004401  
8           True                         -1.237092  
            False                         0.001880  
9           True                         -1.237016  
            False                        -0.010503  
10          True                         -1.240316  
            False                         0.002793  
11          True                         -1.193590  
            False                        -0.005782  
12          True                         -1.228403  
            False                         0.003387  
13          True                         -1.178149  
            False                        -0.020759  
14          True                         -1.212922  
            False                        -0.003940  
15          True                         -1.181714  
            False                        -0.006736

f1_score  roc_auc_score  accuracy_score  \
n_neighbors oversampling                                            
1           True          0.466525       0.701827        0.701312   
            False         0.473287       0.670246        0.823622   
2           True          0.477912       0.698978        0.748294   
            False         0.229658       0.560860        0.824672   
3           True          0.427922       0.678234        0.602625   
            False         0.335107       0.598273        0.820735   
4           True          0.431165       0.675275        0.643832   
            False         0.164262       0.541617        0.822835   
5           True          0.402696       0.653215        0.545144   
            False         0.194435       0.547099        0.814961   
6           True          0.404484       0.652550        0.577428   
            False         0.077672       0.517052        0.815486   
7           True          0.389319       0.637862        0.502625   
            False         0.102107       0.521371        0.812861   
8           True          0.389099       0.635799        0.533858   
            False         0.037315       0.507251        0.813648   
9           True          0.376433       0.620616        0.470866   
            False         0.047646       0.509277        0.813386   
10          True          0.384063       0.630180        0.507874   
            False         0.011073       0.501697        0.813386   
11          True          0.369826       0.611311        0.450394   
            False         0.024580       0.504613        0.813648   
12          True          0.376279       0.620313        0.479265   
            False         0.008295       0.501475        0.813911   
13          True          0.365914       0.605583        0.431496   
            False         0.016635       0.503461        0.814436   
14          True          0.369947       0.611565        0.459055   
            False         0.011112       0.502666        0.814961   
15          True          0.360232       0.596736        0.415486   
            False         0.013994       0.503077        0.814698   

                          explained_variance_score  
n_neighbors oversampling                            
1           True                         -0.738335  
            False                        -0.156131  
2           True                         -0.580306  
            False                        -0.021974  
3           True                         -0.933477  
            False                        -0.116141  
4           True                         -0.923883  
            False                        -0.003620  
5           True                         -0.996959  
            False                        -0.091101  
6           True                         -1.035671  
            False                        -0.025527  
7           True                         -0.988925  
            False                        -0.061649  
8           True                         -1.067391  
            False                        -0.022336  
9           True                         -0.993569  
            False                        -0.029497  
10          True                         -1.045722  
            False                        -0.012188  
11          True                         -0.977004  
            False                        -0.015948  
12          True                         -1.024002  
            False                        -0.006263  
13          True                         -0.935204  
            False                        -0.005225  
14          True                         -1.007300  
            False                         0.002018  
15          True                         -0.920650  
            False                        -0.001523

f1_score  roc_auc_score  accuracy_score  \
n_neighbors oversampling                                            
1           True          0.718377       0.732316        0.732414   
            False         0.711978       0.733584        0.735448   
2           True          0.604366       0.680050        0.691034   
            False         0.548748       0.658381        0.672552   
3           True          0.693575       0.700868        0.699862   
            False         0.674253       0.702600        0.706207   
4           True          0.637677       0.686865        0.694345   
            False         0.585895       0.670882        0.682759   
5           True          0.678755       0.683585        0.681931   
            False         0.655721       0.688139        0.691862   
6           True          0.637283       0.675621        0.680828   
            False         0.592808       0.668095        0.678345   
7           True          0.668606       0.673457        0.672276   
            False         0.640930       0.677361        0.681931   
8           True          0.636327       0.669145        0.673379   
            False         0.581797       0.659881        0.670069   
9           True          0.663387       0.667268        0.665931   
            False         0.630953       0.669914        0.674759   
10          True          0.633242       0.661953        0.665379   
            False         0.572379       0.652238        0.662345   
11          True          0.661832       0.664897        0.663448   
            False         0.618798       0.663991        0.669517   
12          True          0.639576       0.663297        0.665931   
            False         0.585502       0.657241        0.666483   
13          True          0.666153       0.668457        0.666759   
            False         0.622881       0.666462        0.671724   
14          True          0.645296       0.666189        0.668414   
            False         0.576239       0.651249        0.660690   
15          True          0.660555       0.661704        0.659862   
            False         0.612999       0.659581        0.665103   

                          explained_variance_score  
n_neighbors oversampling                            
1           True                         -0.073496  
            False                        -0.059424  
2           True                         -0.140926  
            False                        -0.131944  
3           True                         -0.198300  
            False                        -0.174455  
4           True                         -0.190513  
            False                        -0.154673  
5           True                         -0.265856  
            False                        -0.229166  
6           True                         -0.267791  
            False                        -0.202514  
7           True                         -0.303086  
            False                        -0.265677  
8           True                         -0.304821  
            False                        -0.233077  
9           True                         -0.327815  
            False                        -0.291707  
10          True                         -0.340050  
            False                        -0.265000  
11          True                         -0.335811  
            False                        -0.305343  
12          True                         -0.340002  
            False                        -0.266395  
13          True                         -0.322034  
            False                        -0.297938  
14          True                         -0.328937  
            False                        -0.284609  
15          True                         -0.345863  
            False                        -0.322320

In [20]:
best_knn_1 = knn_results_1.iloc[[
    knn_results_1.reset_index()[DECISION_METRIC].idxmax()]]
best_knn_2 = knn_results_2.iloc[[
    knn_results_2.reset_index()[DECISION_METRIC].idxmax()]]
best_knn_3 = knn_results_3.iloc[[
    knn_results_3.reset_index()[DECISION_METRIC].idxmax()]]

display(best_knn_1, best_knn_2, best_knn_3)

,,f1_score,roc_auc_score,accuracy_score,explained_variance_score
n_neighbors,oversampling,,,,
1,True,0.46468,0.754597,0.740357,-0.791519


,,f1_score,roc_auc_score,accuracy_score,explained_variance_score
n_neighbors,oversampling,,,,
1,True,0.466525,0.701827,0.701312,-0.738335


,,f1_score,roc_auc_score,accuracy_score,explained_variance_score
n_neighbors,oversampling,,,,
1,False,0.711978,0.733584,0.735448,-0.059424


#### 4. Support Vector Classifier

In [23]:
svc_results = hyperparameter_tuning_cv(model='svc', data=X_fine, labels=y_fine.Label, cv_k=5,
                                       params=SVC_PARAMS)

display(svc_results)

f1_score  roc_auc_score  accuracy_score  \
kernel gamma oversampling                                            
linear 0.10  True          0.393986       0.572373        0.630147   
             False         0.002822       0.500456        0.730480   
rbf    0.10  True          0.081298       0.519006        0.738469   
             False         0.028483       0.507232        0.734379   
linear 0.01  True          0.393986       0.572373        0.630147   
             False         0.002822       0.500456        0.730480   
rbf    0.01  True          0.421890       0.602247        0.679220   
             False         0.067001       0.514689        0.735711   

                           explained_variance_score  
kernel gamma oversampling                            
linear 0.10  True                         -0.851848  
             False                        -0.002058  
rbf    0.10  True                          0.000085  
             False                         0.009207  
linear 0.01  True                         -0.851848  
             False                        -0.002058  
rbf    0.01  True                         -0.627410  
             False                        -0.010213

In [24]:
# Best model parameters

best_svc = svc_results.iloc[[svc_results.reset_index()[DECISION_METRIC].idxmax()]]

display(best_svc)

,,,f1_score,roc_auc_score,accuracy_score,explained_variance_score
kernel,gamma,oversampling,,,,
rbf,0.01,True,0.42189,0.602247,0.67922,-0.62741


In [25]:
svc_results_1 = hyperparameter_tuning_cv(
    model='svc', data=X_e_1, labels=y_e_1, cv_k=5, params=SVC_PARAMS)
svc_results_2 = hyperparameter_tuning_cv(
    model='svc', data=X_e_2, labels=y_e_2, cv_k=5, params=SVC_PARAMS)
svc_results_3 = hyperparameter_tuning_cv(
    model='svc', data=X_e_3, labels=y_e_3, cv_k=5, params=SVC_PARAMS)

In [26]:
best_svc_1 = svc_results_1.iloc[[
    svc_results_1.reset_index()[DECISION_METRIC].idxmax()]]
best_svc_2 = svc_results_2.iloc[[
    svc_results_2.reset_index()[DECISION_METRIC].idxmax()]]
best_svc_3 = svc_results_3.iloc[[
    svc_results_3.reset_index()[DECISION_METRIC].idxmax()]]


display(best_svc_1, best_svc_2, best_svc_3)

,,,f1_score,roc_auc_score,accuracy_score,explained_variance_score
kernel,gamma,oversampling,,,,
rbf,0.01,True,0.43055,0.671778,0.824311,-0.416331


,,,f1_score,roc_auc_score,accuracy_score,explained_variance_score
kernel,gamma,oversampling,,,,
rbf,0.01,True,0.282456,0.561125,0.740682,-0.712181


,,,f1_score,roc_auc_score,accuracy_score,explained_variance_score
kernel,gamma,oversampling,,,,
rbf,0.01,True,0.679652,0.695457,0.696,-0.220606


#### 5. Naive Bayes Classifier

In [27]:
nb_results = hyperparameter_tuning_cv(model='naive_bayes', data=X_fine, labels=y_fine.Label, cv_k=5,
                                      params=NAIVE_BAYES_PARAMS)

display(nb_results)

,f1_score,roc_auc_score,accuracy_score,explained_variance_score
oversampling,,,,
True,0.372578,0.559345,0.627199,-0.872985
False,0.335945,0.557551,0.673039,-0.649060


In [28]:
# Best model parameters

best_nb = nb_results.iloc[[nb_results.reset_index()[DECISION_METRIC].idxmax()]]

display(best_nb)

,f1_score,roc_auc_score,accuracy_score,explained_variance_score
oversampling,,,,
True,0.372578,0.559345,0.627199,-0.872985


In [29]:
nb_results_1 = hyperparameter_tuning_cv(
    model='naive_bayes', data=X_e_1, labels=y_e_1, cv_k=5, params=NAIVE_BAYES_PARAMS)
nb_results_2 = hyperparameter_tuning_cv(
    model='naive_bayes', data=X_e_2, labels=y_e_2, cv_k=5, params=NAIVE_BAYES_PARAMS)
nb_results_3 = hyperparameter_tuning_cv(
    model='naive_bayes', data=X_e_3, labels=y_e_3, cv_k=5, params=NAIVE_BAYES_PARAMS)

In [30]:
best_nb_1 = nb_results_1.iloc[[
    nb_results_1.reset_index()[DECISION_METRIC].idxmax()]]
best_nb_2 = nb_results_2.iloc[[
    nb_results_2.reset_index()[DECISION_METRIC].idxmax()]]
best_nb_3 = nb_results_3.iloc[[
    nb_results_3.reset_index()[DECISION_METRIC].idxmax()]]

display(best_nb_1, best_nb_2, best_nb_3)

,f1_score,roc_auc_score,accuracy_score,explained_variance_score
oversampling,,,,
True,0.346586,0.639599,0.684603,-1.241586


,f1_score,roc_auc_score,accuracy_score,explained_variance_score
oversampling,,,,
False,0.220341,0.523959,0.71916,-0.854463


,f1_score,roc_auc_score,accuracy_score,explained_variance_score
oversampling,,,,
True,0.47421,0.589975,0.603034,-0.459593


#### 6. Decision Tree

In [31]:
dt_results = hyperparameter_tuning_cv(model='decision_tree', data=X_fine, labels=y_fine.Label, cv_k=5,
                                      params=DECISION_TREE_PARAMS)

display(dt_results)

f1_score  roc_auc_score  accuracy_score  \
max_depth oversampling                                            
3         True          0.352358       0.540014        0.582406   
          False         0.079716       0.512296        0.727722   
5         True          0.324182       0.545489        0.650214   
          False         0.161130       0.529347        0.726961   
7         True          0.355683       0.561985        0.653828   
          False         0.247548       0.550221        0.726201   

                        explained_variance_score  
max_depth oversampling                            
3         True                         -0.956943  
          False                        -0.084007  
5         True                         -0.735764  
          False                        -0.153994  
7         True                         -0.733147  
          False                        -0.234420

In [32]:
# Best model parameters

best_dt = dt_results.iloc[[dt_results.reset_index()[DECISION_METRIC].idxmax()]]

display(best_dt)

,,f1_score,roc_auc_score,accuracy_score,explained_variance_score
max_depth,oversampling,,,,
7,True,0.355683,0.561985,0.653828,-0.733147


In [33]:
dt_results_1 = hyperparameter_tuning_cv(
    model='decision_tree', data=X_e_1, labels=y_e_1, cv_k=5, params=DECISION_TREE_PARAMS)
dt_results_2 = hyperparameter_tuning_cv(
    model='decision_tree', data=X_e_2, labels=y_e_2, cv_k=5, params=DECISION_TREE_PARAMS)
dt_results_3 = hyperparameter_tuning_cv(
    model='decision_tree', data=X_e_3, labels=y_e_3, cv_k=5, params=DECISION_TREE_PARAMS)

In [34]:
best_dt_1 = dt_results_1.iloc[[
    dt_results_1.reset_index()[DECISION_METRIC].idxmax()]]
best_dt_2 = dt_results_2.iloc[[
    dt_results_2.reset_index()[DECISION_METRIC].idxmax()]]
best_dt_3 = dt_results_3.iloc[[
    dt_results_3.reset_index()[DECISION_METRIC].idxmax()]]

display(best_dt_1, best_dt_2, best_dt_3)

,,f1_score,roc_auc_score,accuracy_score,explained_variance_score
max_depth,oversampling,,,,
5,True,0.354218,0.640811,0.722853,-1.075141


,,f1_score,roc_auc_score,accuracy_score,explained_variance_score
max_depth,oversampling,,,,
7,True,0.289649,0.552053,0.667192,-1.122991


,,f1_score,roc_auc_score,accuracy_score,explained_variance_score
max_depth,oversampling,,,,
5,True,0.611888,0.620609,0.617379,-0.506008


#### 7. Random Forest

In [35]:
rf_results = hyperparameter_tuning_cv(model='random_forest', data=X_fine, labels=y_fine.Label, cv_k=5,
                                      params=RANDOM_FOREST_PARAMS)

display(rf_results)

f1_score  roc_auc_score  accuracy_score  \
max_depth n_estimators oversampling                                            
3         3            True          0.381960       0.549302        0.578602   
                       False         0.036328       0.505784        0.730195   
          5            True          0.385571       0.557820        0.600666   
                       False         0.033596       0.505540        0.730575   
          7            True          0.395751       0.560093        0.585640   
                       False         0.028937       0.504906        0.730766   
5         3            True          0.369195       0.556196        0.622920   
                       False         0.089395       0.512735        0.726010   
          5            True          0.389571       0.567799        0.624631   
                       False         0.052341       0.508067        0.729529   
          7            True          0.388854       0.570193        0.632240   
                       False         0.053488       0.509326        0.731051   
7         3            True          0.369822       0.555165        0.620352   
                       False         0.148387       0.524822        0.725820   
          5            True          0.358044       0.553805        0.634712   
                       False         0.111833       0.519271        0.729339   
          7            True          0.372928       0.571052        0.662197   
                       False         0.108127       0.521105        0.733619   

                                     explained_variance_score  
max_depth n_estimators oversampling                            
3         3            True                         -1.025608  
                       False                        -0.029842  
          5            True                         -0.956936  
                       False                        -0.024882  
          7            True                         -0.993744  
                       False                        -0.019853  
5         3            True                         -0.890431  
                       False                        -0.104272  
          5            True                         -0.874791  
                       False                        -0.047259  
          7            True                         -0.838527  
                       False                        -0.035949  
7         3            True                         -0.907250  
                       False                        -0.153261  
          5            True                         -0.850104  
                       False                        -0.095608  
          7            True                         -0.714624  
                       False                        -0.059459

In [36]:
# Best model parameters

best_rf = rf_results.iloc[[rf_results.reset_index()[DECISION_METRIC].idxmax()]]

display(best_rf)

,,,f1_score,roc_auc_score,accuracy_score,explained_variance_score
max_depth,n_estimators,oversampling,,,,
3,7,True,0.395751,0.560093,0.58564,-0.993744


In [37]:
rf_results_1 = hyperparameter_tuning_cv(
    model='random_forest', data=X_e_1, labels=y_e_1, cv_k=5, params=RANDOM_FOREST_PARAMS)
rf_results_2 = hyperparameter_tuning_cv(
    model='random_forest', data=X_e_2, labels=y_e_2, cv_k=5, params=RANDOM_FOREST_PARAMS)
rf_results_3 = hyperparameter_tuning_cv(
    model='random_forest', data=X_e_3, labels=y_e_3, cv_k=5, params=RANDOM_FOREST_PARAMS)

In [38]:
best_rf_1 = rf_results_1.iloc[[
    rf_results_1.reset_index()[DECISION_METRIC].idxmax()]]
best_rf_2 = rf_results_2.iloc[[
    rf_results_2.reset_index()[DECISION_METRIC].idxmax()]]
best_rf_3 = rf_results_3.iloc[[
    rf_results_3.reset_index()[DECISION_METRIC].idxmax()]]

display(best_rf_1, best_rf_2, best_rf_3)

,,,f1_score,roc_auc_score,accuracy_score,explained_variance_score
max_depth,n_estimators,oversampling,,,,
7,5,True,0.395102,0.66012,0.781524,-0.722536


,,,f1_score,roc_auc_score,accuracy_score,explained_variance_score
max_depth,n_estimators,oversampling,,,,
7,7,True,0.276032,0.555346,0.726772,-0.801158


,,,f1_score,roc_auc_score,accuracy_score,explained_variance_score
max_depth,n_estimators,oversampling,,,,
7,7,False,0.591165,0.638466,0.643586,-0.412352


#### 8. Gradient Boosting

In [39]:
gb_results = hyperparameter_tuning_cv(model='gradient_boosting', data=X_fine, labels=y_fine.Label, cv_k=5,
                                      params=GRADIENT_BOOSTING_PARAMS)

display(gb_results)

f1_score  roc_auc_score  accuracy_score  \
max_depth n_estimators oversampling                                            
3         3            True          0.373577       0.563819        0.639372   
                       False         0.000000       0.500000        0.730480   
          5            True          0.371229       0.567133        0.650214   
                       False         0.001351       0.500339        0.730670   
          7            True          0.360229       0.561771        0.652211   
                       False         0.002106       0.500527        0.730766   
5         3            True          0.344987       0.569462        0.693676   
                       False         0.000000       0.500000        0.730480   
          5            True          0.334492       0.569156        0.704042   
                       False         0.004898       0.501098        0.730956   
          7            True          0.338157       0.572908        0.709843   
                       False         0.019387       0.504014        0.731812   
7         3            True          0.354732       0.566570        0.674465   
                       False         0.000000       0.499806        0.730195   
          5            True          0.346100       0.572471        0.699477   
                       False         0.029726       0.506058        0.732287   
          7            True          0.343053       0.574217        0.708512   
                       False         0.066446       0.514218        0.735045   

                                     explained_variance_score  
max_depth n_estimators oversampling                            
3         3            True                         -0.815863  
                       False                         0.000000  
          5            True                         -0.769336  
                       False                         0.000414  
          7            True                         -0.762295  
                       False                         0.000664  
5         3            True                         -0.528224  
                       False                         0.000000  
          5            True                         -0.456169  
                       False                         0.000070  
          7            True                         -0.421937  
                       False                        -0.004109  
7         3            True                         -0.643234  
                       False                        -0.002244  
          5            True                         -0.491999  
                       False                        -0.008016  
          7            True                         -0.433170  
                       False                        -0.014714

In [40]:
# Best model parameters

best_gb = gb_results.iloc[[gb_results.reset_index()[DECISION_METRIC].idxmax()]]

display(best_gb)

,,,f1_score,roc_auc_score,accuracy_score,explained_variance_score
max_depth,n_estimators,oversampling,,,,
3,3,True,0.373577,0.563819,0.639372,-0.815863


In [41]:
gb_results_1 = hyperparameter_tuning_cv(
    model='gradient_boosting', data=X_e_1, labels=y_e_1, cv_k=5, params=GRADIENT_BOOSTING_PARAMS)
gb_results_2 = hyperparameter_tuning_cv(
    model='gradient_boosting', data=X_e_2, labels=y_e_2, cv_k=5, params=GRADIENT_BOOSTING_PARAMS)
gb_results_3 = hyperparameter_tuning_cv(
    model='gradient_boosting', data=X_e_3, labels=y_e_3, cv_k=5, params=GRADIENT_BOOSTING_PARAMS)

In [42]:
best_gb_1 = gb_results_1.iloc[[
    gb_results_1.reset_index()[DECISION_METRIC].idxmax()]]
best_gb_2 = gb_results_2.iloc[[
    gb_results_2.reset_index()[DECISION_METRIC].idxmax()]]
best_gb_3 = gb_results_3.iloc[[
    gb_results_3.reset_index()[DECISION_METRIC].idxmax()]]
display(best_gb_1, best_gb_2, best_gb_3)

,,,f1_score,roc_auc_score,accuracy_score,explained_variance_score
max_depth,n_estimators,oversampling,,,,
5,7,True,0.414945,0.67587,0.787682,-0.665224


,,,f1_score,roc_auc_score,accuracy_score,explained_variance_score
max_depth,n_estimators,oversampling,,,,
5,7,True,0.289367,0.56312,0.735696,-0.743254


,,,f1_score,roc_auc_score,accuracy_score,explained_variance_score
max_depth,n_estimators,oversampling,,,,
5,7,False,0.626027,0.677945,0.68469,-0.227881


### Results

In [43]:
#display('logistic', best_log)
#display('lda', best_lda)
#display('knn', best_knn)
#display('svc', best_svc)
#display('naive_bayes', best_nb)
#display('decision_tree', best_dt)
#display('random_forest', best_rf)
#display('gradient_boosting', best_gb)

In [44]:
weighted_average = (np.array([len(X_e_1), len(X_e_2), len(X_e_3)]) / len(X_e))

e_log = np.sum(pd.concat([best_log_1, best_log_2, best_log_3]
                         ).roc_auc_score.values * weighted_average)
e_lda = np.sum(pd.concat([best_lda_1, best_lda_2, best_lda_3]
                         ).roc_auc_score.values * weighted_average)
e_knn = np.sum(pd.concat([best_knn_1, best_knn_2, best_knn_3]
                         ).roc_auc_score.values * weighted_average)
e_svc = np.sum(pd.concat([best_svc_1, best_svc_2, best_svc_3]
                         ).roc_auc_score.values * weighted_average)
e_nb = np.sum(pd.concat([best_nb_1, best_nb_2, best_nb_3]
                        ).roc_auc_score.values * weighted_average)
e_dt = np.sum(pd.concat([best_dt_1, best_dt_2, best_dt_3]
                        ).roc_auc_score.values * weighted_average)
e_rf = np.sum(pd.concat([best_rf_1, best_rf_2, best_rf_3]
                        ).roc_auc_score.values * weighted_average)
e_gb = np.sum(pd.concat([best_gb_1, best_gb_2, best_gb_3]
                        ).roc_auc_score.values * weighted_average)

In [45]:
# Results expert features
# First row is Expert 1, second Expert 2, third Expert 3
#display('logistic', pd.concat([best_log_1,best_log_2,best_log_3]))
#display('lda', pd.concat([best_lda_1,best_lda_2,best_lda_3]))
display('knn', pd.concat([best_knn_1, best_knn_2, best_knn_3]))
#display('svc', pd.concat([best_svc_1,best_svc_2,best_svc_3]))
#display('naive_bayes', pd.concat([best_nb_1,best_nb_2,best_nb_3]))
#display('decision_tree', pd.concat([best_dt_1,best_dt_2,best_dt_3]))
#display('random_forest', pd.concat([best_rf_1,best_rf_2,best_rf_3]))
#display('gradient_boosting', pd.concat([best_gb_1,best_gb_2,best_gb_3]))

'knn'

f1_score  roc_auc_score  accuracy_score  \
n_neighbors oversampling                                            
1           True          0.464680       0.754597        0.740357   
            True          0.466525       0.701827        0.701312   
            False         0.711978       0.733584        0.735448   

                          explained_variance_score  
n_neighbors oversampling                            
1           True                         -0.791519  
            True                         -0.738335  
            False                        -0.059424

In [46]:
results = pd.DataFrame(data={'models': ['logistic', 'lda', 'knn', 'svc', 'naive_bayes', 'decision_tree',
                                        'random_forest', 'gradient_boosting'],
                             'auc_best': [best_log.roc_auc_score.values[0], best_lda.roc_auc_score.values[0],
                                          best_knn.roc_auc_score.values[0], best_svc.roc_auc_score.values[0],
                                          best_nb.roc_auc_score.values[0], best_dt.roc_auc_score.values[0],
                                          best_rf.roc_auc_score.values[0], best_gb.roc_auc_score.values[0]],
                             'auc_expert_weighted': [e_log, e_lda, e_knn, e_svc, e_nb, e_dt, e_rf, e_gb]})

display(results)

,models,auc_best,auc_expert_weighted
0,logistic,0.574983,0.616548
1,lda,0.576162,0.614492
2,knn,0.683665,0.728246
3,svc,0.602247,0.639872
4,naive_bayes,0.559345,0.580620
5,decision_tree,0.561985,0.601707
6,random_forest,0.560093,0.614716
7,gradient_boosting,0.563819,0.635758


### Conclusions


In [47]:
# Save dataframe
# results.to_pickle("results_fine_metadata.pkl")

In [48]:
results.mean(axis=0)

auc_best               0.585288
auc_expert_weighted    0.628995
dtype: float64

In [49]:
results.loc[results['auc_best'].argmax()]

models                      knn
auc_best               0.683665
auc_expert_weighted    0.728246
Name: 2, dtype: object

In [50]:
results.loc[results['auc_expert_weighted'].argmax()]

models                      knn
auc_best               0.683665
auc_expert_weighted    0.728246
Name: 2, dtype: object